### Assignment : Week 3
## Efficiently finding optimal policies in MABs

In this assignment, we will work with Multi Armed Bandit environments, and try to find the best policies using different strategies to minimize the total regret.

The aim of this exercise is to code agents capable of understanding the underlying probability distributions of the environment and finding the most optimal actions as early as possible.

You can start this assignment during/after reading Grokking Ch-4.

Let's get started!

In [53]:
# importing necessary stuff
import numpy as np
from pprint import pprint
from tqdm.notebook import tqdm

# if you want to use envs from Gym, import it
import gym, gym_bandits

from math import log, sqrt, log10

from bandits import Bandit

Let's make a simple **2-armed Bernoulli** bandit.

If you want a cleaner code, you can implement Bandits using `class` in Python.

We have included sample code for this in `bandits.py` which you can take/import.

In [17]:
# generating the underlying probability distribution

probs = np.random.random(2)

In [11]:
# our MDP is a function which takes an action and returns a reward

def mab_2_env(action):
    gen = np.random.random()

    # for bernoulli bandits, the reward is 1 if the random number is less than the probability of success, else 0
    return gen < probs[action]

Let's now make a template for testing different strategies.

Feel free to modify this code.

In [12]:
# strategy function takes in the environment function, number of actions, and a selector function
# it also takes in the number of episodes to run the strategy for (higher episodes = more accurate Q values)

def test_strategy(env, n_actions, selector, n_episodes = 1000):
    
    # initialize Q and N to 0s
    Q = np.zeros(n_actions)
    N = np.zeros(n_actions)

    # loop for n_episodes
    for e in tqdm(range(n_episodes)):
        
        # selector function takes in current Q and returns an action
        # modify the selector function according to the strategy
        action = selector(Q)

        # get the reward from the environment
        reward = env(action)

        # update N and Q
        N[action] += 1
        Q[action] += (reward - Q[action])/N[action]

    # return the best action
    return np.argmax(Q)

Implementing the simplest selector using pure-exploration strategy.

In [18]:
# selector returns a random action
random_selector = lambda Q : np.random.randint(len(Q))

Let's test this strategy.

In [19]:
test_strategy(mab_2_env, 2, random_selector)

  0%|          | 0/1000 [00:00<?, ?it/s]

0

As you can see, it returns the optimal action. Let's check if that's indeed true.

We can do that by revealing the actual `probs` distribution.

In [20]:
probs

array([0.39186151, 0.16238667])

As expected, our pure exploration strategy does indeed return the optimal action for this Bernoulli bandit. 

You can try generating new bandits with different `probs` and try out the same.

With all this in place, here's what you have to do -

Recall that, the regret $\mathcal{T}$ is given by,

$$\mathcal{T}=\sum  _{e=1} ^{E} \mathbb{E} \left[ v_* - q_* \left( A_e \right) \right]$$

We can only calculate it when we have the $v_*$ and $q_*$ functions known beforehand. Since we are making the MDPs from scratch, that's not an issue for us right now.

But remember, in real-life problems, these functions are not known. Hence we must be aware of multiple policy finding strategies and try the one which gives best results fastest.

### Todo 0

Implement the calculation of the total regret $\mathcal{T}$ for your strategy.

To do this, you will need to store the rewards obtained each episode. Modify the `strategy` function accordingly.

In [21]:
# function to calculate total regret

def regret(rewards, probs):

    return len(rewards) * np.max(probs) - sum(rewards)

### Todo 1

Now, let's implement some other selection strategies and compare their regret with the simple exploration strategy.

Note that some of these strategies involve hyperparameter(s) which need to be manually adjusted. You have to play around with the values and see which one gives you best results.

This is known as "hyperparameter tuning" and is quite commonly done while working with complex models (including neural networks). Personally, you should try out some natural values (including the ones given in the book) along with some extreme values where it is easy to manually verify the correctness of your strategy.

In [65]:
# epsilon-greedy strategy
# Already implemented for you coz I am nice (Thanks UwU -Nirav)

def epsilon_greedy(Q, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(len(Q))
    else:
        return np.argmax(Q)

In [25]:
# exponentially decaying epsilon greedy strategy

def exponentially_decaying_epsilon_greedy(Q, epsilon, episode, gamma = 0.99):
    
    epsilon *= gamma**episode
    if np.random.random() < epsilon:
        return np.random.randint(len(Q)), epsilon
    return np.argmax(Q) + epsilon

In [28]:
# softmax action selection strategy

def softmax_strategy(Q, temperature):
    
    assert temperature > 0

    values = np.exp(Q / temperature)
    probabilities = values / np.sum(values)
    action = np.random.choice(len(Q), p=probabilities)

    return action


In [29]:
# upper confidence bound strategy

def ucb(Q, N, c):
    
    total_rounds = sum(N)
    ucb_values = [Q[i] + c * sqrt(log(total_rounds) / N[i]) if N[i] >= 1 else Q[i] for i in range(len(Q))]
    chosen_action = np.argmax(ucb_values) + 1 

    return chosen_action

In [30]:
# thompson sampling strategy

def thompson_sampling(Q, N, alpha, beta):
    
    samples = np.random.normal(loc=Q, scale=alpha / (sqrt(N) + beta))
    chosen_action = np.argmax(samples) + 1 

    return chosen_action

### Todo 2

Run each strategy for 2-armed bandit environment and compare the total regrets.

You can also try plotting the regret vs episode graph and check if it matches the expected result from Grokking

In [84]:
# import plotting libraries

import matplotlib.pyplot as plt

In [58]:
bernoulli_bandit = Bandit(10, "Bernoulli")
bernoulli_bandit.probs
bandit = bernoulli_bandit

In [112]:
def epsilon_greedy_implementation(epsilon=0.1, gamma=0.9, num_episodes=10000):
    global bandit
    Q = np.zeros(bandit.N)
    N = np.zeros(bandit.N)

    for episode in range(num_episodes):
        choice = epsilon_greedy(Q, epsilon)
        reward = bandit.choose(choice)
        regret = bandit.optimal - reward
        bandit.regret += regret

        Q[choice-1] = (Q[choice-1] * N[choice-1] + gamma * reward) / (N[choice-1] + 1)
        N[choice-1] += 1

    return np.argmax(Q)

In [74]:
def implement_decaying_epsilon_greedy(epsilon, gamma=0.9, num_episodes = 10000): ## Credit to Aditya Neeraje, helped me tremendously
    global bandit
    min_to_max_ratio = 0.01
    max_epsilon = epsilon
    min_epsilon = min_to_max_ratio*epsilon
    decay_rate = 0.05
    decay_episodes = int(num_episodes*decay_rate)
    remaining_episodes = num_episodes - decay_episodes
    epsilons = 0.01
    epsilons /= np.logspace(log10(min_to_max_ratio), 0, decay_episodes)
    epsilons *= max_epsilon - min_epsilon
    epsilons += min_epsilon
    epsilons = np.pad(epsilons, (0, remaining_episodes), mode='constant', constant_values=min_epsilon)
    bandit.regret = 0
    gamma_power = gamma
    Q = np.zeros(bandit.N)
    N = np.zeros(bandit.N)
    for episode in range(num_episodes):
        choice = epsilon_greedy(Q, epsilons[episode])
        Q[choice-1] = (Q[choice-1]*N[choice-1] + gamma * bandit.choose(choice))/(N[choice-1] + 1)
        N[choice-1] += 1
        gamma_power *= gamma
        
    return np.argmax(Q)

In [75]:
def implement_softmax(temperature, gamma=0.9, num_episodes = 10000):
    global bandit
    bandit.regret = 0
    gamma_power = gamma
    Q = np.zeros(bandit.N)
    N = np.zeros(bandit.N)
    for episode in range(num_episodes):
        choice = softmax_strategy(Q, temperature)
        Q[choice-1] = (Q[choice-1]*N[choice-1] + gamma * bandit.evaluate_choice(choice))/(N[choice-1] + 1)
        N[choice-1] += 1
        gamma_power *= gamma
    return np.argmax(Q)

In [76]:
def implement_ucb(c, gamma=0.9, num_episodes = 10000):
    global bandit
    bandit.regret = 0
    gamma_power = gamma
    Q = np.zeros(bandit.N)
    N = np.ones(bandit.N)
    for episode in range(num_episodes):
        choice = ucb(Q, N, c)
        Q[choice-1] = (Q[choice-1]*N[choice-1] + gamma * bandit.evaluate_choice(choice))/(N[choice-1] + 1)
        N[choice-1] += 1
        gamma_power *= gamma
    return np.argmax(Q)

In [118]:
def implement_thompson_sampling(alpha=1, beta=0, gamma = 0.9, num_episodes = 10000):
    global bandit
    bandit.regret = 0
    gamma_power = gamma
    Q = np.zeros(bandit.N)
    N = np.zeros(bandit.N)
    for episode in range(num_episodes):
        choice = thompson_sampling(Q, N, alpha, beta)
        Q[choice-1] = (Q[choice-1]*N[choice-1] + gamma * bandit.evaluate_choice(choice))/(N[choice-1] + 1)
        N[choice-1] += 1
        gamma_power *= gamma
    return np.argmax(Q)

### Todo 3

The 2-armed bandits might be too simple for us to actually see substantial difference in the regret of these strategies. 

Let's now create a more complicated bandit environment and replicate our results on it.

We will now implement a 10-armed Gaussian bandit. 

As required, it will have possible actions and each action will generate a reward sampled from a Gaussian distribution.

Hence, each "arm" will have a randomly generated $\mu$ and $\sigma$, and the rewards will be generated with probabilities following the $\mathcal{N}(\mu, \sigma^2)$ distribution. 

In [53]:
# 10 arm gaussian bandit

# generate the means for each arm
means = []

# generate the variance for each arm
variances = []

In [54]:
# our MDP is a again function which takes an action and returns a reward

def mab_10_env(action):

    # for gaussian bandits, the reward is generated from a normal distribution
    raise NotImplementedError()

### Todo 4

Test the different strategies on the 10-armed gaussian bandit and verify your results.